In [1]:
import os
import glob
import numpy as np
import pandas as pd
import SimpleITK as sitk
from medpy.metric import binary

from scipy.ndimage import label



# 保存nifti函数
def save_array_to_nii(mask_array, nii_template_path, nii_save_path):
    sitk_image_object = sitk.ReadImage(nii_template_path)
    output_spacing = sitk_image_object.GetSpacing()
    output_direction = sitk_image_object.GetDirection()
    output_origin = sitk_image_object.GetOrigin()

    nrrd_output = sitk.GetImageFromArray(mask_array.astype(np.uint8))
    nrrd_output.SetSpacing(output_spacing)
    nrrd_output.SetDirection(output_direction)
    nrrd_output.SetOrigin(output_origin)

    sitk.WriteImage(nrrd_output, nii_save_path, True)  # True 表示压缩保存
    print(f"Saved: {nii_save_path}")

def largest_connected_component(binary_mask: np.ndarray) -> np.ndarray:
    """
    保留二值mask中最大的连通域
    输入: binary_mask (bool ndarray)
    输出: 只包含最大连通区域的bool ndarray
    """
    sitk_mask = sitk.GetImageFromArray(binary_mask.astype(np.uint8))
    cc_filter = sitk.ConnectedComponentImageFilter()
    labeled = cc_filter.Execute(sitk_mask)
    stats = sitk.LabelShapeStatisticsImageFilter()
    stats.Execute(labeled)

    if stats.GetNumberOfLabels() == 0:
        # 没有连通区域，返回空mask
        return np.zeros_like(binary_mask, dtype=bool)

    # 找最大连通区域标签
    largest_label = max(stats.GetLabels(), key=lambda l: stats.GetPhysicalSize(l))
    largest_cc = labeled == largest_label
    largest_cc_arr = sitk.GetArrayFromImage(largest_cc)
    return largest_cc_arr.astype(bool)


In [22]:
import os
import glob
import SimpleITK as sitk
import numpy as np
from medpy.metric import binary
import pandas as pd

np.bool = bool  # 修复numpy.bool警告

# 假设 largest_connected_component 函数已定义

seg_dir = '/home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw_910_curated'
# label_dir = '/home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_test_data/test_sam2_1slicepromt_seg_in_nii_2class'
label_dir = '/home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw_912_curated'


all_classes = [1, 2]
records = []

seg_files = sorted(glob.glob(os.path.join(seg_dir, '*.nii.gz')))

for seg_path in seg_files:
    seg_name = os.path.basename(seg_path)
    pid = '_'.join(seg_name.split('_')[:3])
    if 'ID' in pid:
        pid = pid.replace('water_L4L5', '')
    if '_T' in pid or 'word' in pid:
        pid = '_'.join(seg_name.split('_')[:2])
    if pid[0] == 's':
        pid = pid.split('_')[0]    
        
    pid = pid.replace('_L4L5', '')   


    label_candidates = glob.glob(os.path.join(label_dir, f"{pid}*.nii.gz"))
    if not label_candidates:
        print(f"❌ No ground truth label found for {pid}")
        continue

    label_path = label_candidates[0]

    seg_arr = sitk.GetArrayFromImage(sitk.ReadImage(seg_path))
    label_arr = sitk.GetArrayFromImage(sitk.ReadImage(label_path))

    dsc_values = {}
    decrease_values = {}

    for cls in all_classes:
        seg_bin = (seg_arr == cls)
        label_bin = (label_arr == cls)

        seg_bin_lcc = largest_connected_component(seg_bin)
        label_bin_lcc = largest_connected_component(label_bin)

        slice_areas = np.array([np.sum(seg_bin[z, :, :]) for z in range(seg_bin.shape[0])])
        ratios = slice_areas[1:] / (slice_areas[:-1] + 1e-8)  # 防止除零
        # 1.5 for 1st, 1.25 for 2nd
        decreased = np.all(ratios <= 1.25)  # 每片面积不超过前一片的1.5 OR 1.25倍

        if np.sum(label_bin_lcc) == 0:
            dsc = np.nan
            print(f"⚠️ {pid} - Class {cls} not present in ground truth. DSC: NaN")
        else:
            try:
                dsc = binary.dc(seg_bin_lcc, label_bin_lcc)
                print(f"✅ {pid} - Class {cls} - DSC: {dsc:.4f} - Decreased: {decreased}")
            except Exception as e:
                dsc = np.nan
                print(f"❌ Error computing DSC for {pid} - Class {cls}: {e}")

        dsc_values[cls] = dsc
        decrease_values[cls] = decreased

    mean_dsc = np.nanmean(list(dsc_values.values()))

    records.append({
        "pid": pid,
        "dsc_class_1": round(float(dsc_values.get(1, np.nan)), 4) if not np.isnan(dsc_values.get(1, np.nan)) else None,
        "dsc_class_2": round(float(dsc_values.get(2, np.nan)), 4) if not np.isnan(dsc_values.get(2, np.nan)) else None,
        "mean_dsc": round(float(mean_dsc), 4) if not np.isnan(mean_dsc) else None,
        "decrease_class_1": decrease_values.get(1, False),
        "decrease_class_2": decrease_values.get(2, False),
    })



✅ 0001_T1 - Class 1 - DSC: 0.9813 - Decreased: True
✅ 0001_T1 - Class 2 - DSC: 0.9782 - Decreased: True
✅ 0001_T2 - Class 1 - DSC: 0.9794 - Decreased: True
✅ 0001_T2 - Class 2 - DSC: 0.9792 - Decreased: True
✅ 0002_T1 - Class 1 - DSC: 0.9773 - Decreased: True
✅ 0002_T1 - Class 2 - DSC: 0.9848 - Decreased: True
✅ 0002_T2 - Class 1 - DSC: 0.9816 - Decreased: True
✅ 0002_T2 - Class 2 - DSC: 0.9849 - Decreased: True
✅ 0003_T1 - Class 1 - DSC: 0.9952 - Decreased: True
✅ 0003_T1 - Class 2 - DSC: 0.9949 - Decreased: True
✅ 0003_T2 - Class 1 - DSC: 0.9949 - Decreased: True
✅ 0003_T2 - Class 2 - DSC: 0.9947 - Decreased: True
✅ 0004_T1 - Class 1 - DSC: 0.9877 - Decreased: True
✅ 0004_T1 - Class 2 - DSC: 0.9871 - Decreased: True
✅ 0004_T2 - Class 1 - DSC: 0.9942 - Decreased: True
✅ 0004_T2 - Class 2 - DSC: 0.9945 - Decreased: True
✅ 0005_T1 - Class 1 - DSC: 0.9828 - Decreased: True
✅ 0005_T1 - Class 2 - DSC: 0.9809 - Decreased: True
✅ 0005_T2 - Class 1 - DSC: 0.9865 - Decreased: True
✅ 0005_T2 - 

✅ 0041_T1 - Class 4 - DSC: 0.9495
✅ 0041_T2 - Class 1 - DSC: 0.9718
✅ 0041_T2 - Class 2 - DSC: 0.9602
✅ 0041_T2 - Class 3 - DSC: 0.9609
✅ 0041_T2 - Class 4 - DSC: 0.9622
✅ 0043_T1 - Class 1 - DSC: 0.9571
✅ 0043_T1 - Class 2 - DSC: 0.9506
✅ 0043_T1 - Class 3 - DSC: 0.9628
✅ 0043_T1 - Class 4 - DSC: 0.9454
✅ 0043_T2 - Class 1 - DSC: 0.9561
✅ 0043_T2 - Class 2 - DSC: 0.9576
✅ 0043_T2 - Class 3 - DSC: 0.9163
✅ 0043_T2 - Class 4 - DSC: 0.9302
✅ 0044_T1 - Class 1 - DSC: 0.9575
✅ 0044_T1 - Class 2 - DSC: 0.9310
✅ 0044_T1 - Class 3 - DSC: 0.9238
✅ 0044_T1 - Class 4 - DSC: 0.9475
✅ 0044_T2 - Class 1 - DSC: 0.9594
✅ 0044_T2 - Class 2 - DSC: 0.9272
✅ 0044_T2 - Class 3 - DSC: 0.7998
✅ 0044_T2 - Class 4 - DSC: 0.8954
✅ 0046_T1 - Class 1 - DSC: 0.9552
✅ 0046_T1 - Class 2 - DSC: 0.9425
✅ 0046_T1 - Class 3 - DSC: 0.9618
✅ 0046_T1 - Class 4 - DSC: 0.9673
✅ 0046_T2 - Class 1 - DSC: 0.9487
✅ 0046_T2 - Class 2 - DSC: 0.9277
✅ 0046_T2 - Class 3 - DSC: 0.9367
✅ 0046_T2 - Class 4 - DSC: 0.9529
✅ 0047_T1 - Cl

✅ 0088_T1 - Class 4 - DSC: 0.9626
✅ 0088_T2 - Class 1 - DSC: 0.9347
✅ 0088_T2 - Class 2 - DSC: 0.9338
✅ 0088_T2 - Class 3 - DSC: 0.9313
✅ 0088_T2 - Class 4 - DSC: 0.9572
✅ 0089_T1 - Class 1 - DSC: 0.9486
✅ 0089_T1 - Class 2 - DSC: 0.9631
✅ 0089_T1 - Class 3 - DSC: 0.9563
✅ 0089_T1 - Class 4 - DSC: 0.9548
✅ 0089_T2 - Class 1 - DSC: 0.9479
✅ 0089_T2 - Class 2 - DSC: 0.9693
✅ 0089_T2 - Class 3 - DSC: 0.9473
✅ 0089_T2 - Class 4 - DSC: 0.9495
✅ 0090_T1 - Class 1 - DSC: 0.9451
✅ 0090_T1 - Class 2 - DSC: 0.9101
✅ 0090_T1 - Class 3 - DSC: 0.9155
✅ 0090_T1 - Class 4 - DSC: 0.9248
✅ 0090_T2 - Class 1 - DSC: 0.9509
✅ 0090_T2 - Class 2 - DSC: 0.9226
✅ 0090_T2 - Class 3 - DSC: 0.9390
✅ 0090_T2 - Class 4 - DSC: 0.9500
✅ 0092_T1 - Class 1 - DSC: 0.9414
✅ 0092_T1 - Class 2 - DSC: 0.9454
✅ 0092_T1 - Class 3 - DSC: 0.9576
✅ 0092_T1 - Class 4 - DSC: 0.9527
✅ 0092_T2 - Class 1 - DSC: 0.9444
✅ 0092_T2 - Class 2 - DSC: 0.9434
✅ 0092_T2 - Class 3 - DSC: 0.9405
✅ 0092_T2 - Class 4 - DSC: 0.9430
✅ 0093_T1 - Cl

✅ 0132_T2 - Class 1 - DSC: 0.9338
✅ 0132_T2 - Class 2 - DSC: 0.9314
✅ 0132_T2 - Class 3 - DSC: 0.9578
✅ 0132_T2 - Class 4 - DSC: 0.9604
✅ 0134_T1 - Class 1 - DSC: 0.9383
✅ 0134_T1 - Class 2 - DSC: 0.9568
✅ 0134_T1 - Class 3 - DSC: 0.9205
✅ 0134_T1 - Class 4 - DSC: 0.9050
✅ 0134_T2 - Class 1 - DSC: 0.9300
✅ 0134_T2 - Class 2 - DSC: 0.9145
✅ 0134_T2 - Class 3 - DSC: 0.8696
✅ 0134_T2 - Class 4 - DSC: 0.9267
✅ 0136_T1 - Class 1 - DSC: 0.9235
✅ 0136_T1 - Class 2 - DSC: 0.9043
✅ 0136_T1 - Class 3 - DSC: 0.9332
✅ 0136_T1 - Class 4 - DSC: 0.9226
✅ 0136_T2 - Class 1 - DSC: 0.9681
✅ 0136_T2 - Class 2 - DSC: 0.9682
✅ 0136_T2 - Class 3 - DSC: 0.9691
✅ 0136_T2 - Class 4 - DSC: 0.9486
✅ 0137_T1 - Class 1 - DSC: 0.9263
✅ 0137_T1 - Class 2 - DSC: 0.9207
✅ 0137_T1 - Class 3 - DSC: 0.9341
✅ 0137_T1 - Class 4 - DSC: 0.9363
✅ 0137_T2 - Class 1 - DSC: 0.9602
✅ 0137_T2 - Class 2 - DSC: 0.9504
✅ 0137_T2 - Class 3 - DSC: 0.9447
✅ 0137_T2 - Class 4 - DSC: 0.9623
✅ 0138_T1 - Class 1 - DSC: 0.8937
✅ 0138_T1 - Cl

✅ 0176_T2 - Class 3 - DSC: 0.9447
✅ 0176_T2 - Class 4 - DSC: 0.9275
✅ 0177_T1 - Class 1 - DSC: 0.9567
✅ 0177_T1 - Class 2 - DSC: 0.9517
✅ 0177_T1 - Class 3 - DSC: 0.9655
✅ 0177_T1 - Class 4 - DSC: 0.9658
✅ 0177_T2 - Class 1 - DSC: 0.9623
✅ 0177_T2 - Class 2 - DSC: 0.9571
✅ 0177_T2 - Class 3 - DSC: 0.9461
✅ 0177_T2 - Class 4 - DSC: 0.9596
✅ 0178_T1 - Class 1 - DSC: 0.9443
✅ 0178_T1 - Class 2 - DSC: 0.9536
✅ 0178_T1 - Class 3 - DSC: 0.9049
✅ 0178_T1 - Class 4 - DSC: 0.9221
✅ 0178_T2 - Class 1 - DSC: 0.9520
✅ 0178_T2 - Class 2 - DSC: 0.9453
✅ 0178_T2 - Class 3 - DSC: 0.9509
✅ 0178_T2 - Class 4 - DSC: 0.9544
✅ 0179_T1 - Class 1 - DSC: 0.9398
✅ 0179_T1 - Class 2 - DSC: 0.9317
✅ 0179_T1 - Class 3 - DSC: 0.9408
✅ 0179_T1 - Class 4 - DSC: 0.9113
✅ 0181_T1 - Class 1 - DSC: 0.9462
✅ 0181_T1 - Class 2 - DSC: 0.8914
✅ 0181_T1 - Class 3 - DSC: 0.8729
✅ 0181_T1 - Class 4 - DSC: 0.8934
✅ 0181_T2 - Class 1 - DSC: 0.9541
✅ 0181_T2 - Class 2 - DSC: 0.9078
✅ 0181_T2 - Class 3 - DSC: 0.9101
✅ 0181_T2 - Cl

...

✅ 0393_T2 - Class 1 - DSC: 0.9662
✅ 0393_T2 - Class 2 - DSC: 0.9679
✅ 0393_T2 - Class 3 - DSC: 0.9497
✅ 0393_T2 - Class 4 - DSC: 0.9506
✅ 0394_T1 - Class 1 - DSC: 0.9449
✅ 0394_T1 - Class 2 - DSC: 0.9365
✅ 0394_T1 - Class 3 - DSC: 0.9496
✅ 0394_T1 - Class 4 - DSC: 0.9579
✅ 0394_T2 - Class 1 - DSC: 0.9448
✅ 0394_T2 - Class 2 - DSC: 0.9415
✅ 0394_T2 - Class 3 - DSC: 0.9389
✅ 0394_T2 - Class 4 - DSC: 0.9521
✅ 0396_T1 - Class 1 - DSC: 0.9352
✅ 0396_T1 - Class 2 - DSC: 0.9577
✅ 0396_T1 - Class 3 - DSC: 0.9422
✅ 0396_T1 - Class 4 - DSC: 0.9358
✅ 0396_T2 - Class 1 - DSC: 0.9483
✅ 0396_T2 - Class 2 - DSC: 0.9544
✅ 0396_T2 - Class 3 - DSC: 0.9569
✅ 0396_T2 - Class 4 - DSC: 0.9586
✅ 0400_T1 - Class 1 - DSC: 0.9605
✅ 0400_T1 - Class 2 - DSC: 0.9705
✅ 0400_T1 - Class 3 - DSC: 0.9695
✅ 0400_T1 - Class 4 - DSC: 0.9656
✅ 0400_T2 - Class 1 - DSC: 0.9734
✅ 0400_T2 - Class 2 - DSC: 0.9555
✅ 0400_T2 - Class 3 - DSC: 0.9686
✅ 0400_T2 - Class 4 - DSC: 0.9675
✅ 0401_T1 - Class 1 - DSC: 0.8475
✅ 0401_T1 - Cl

✅ 0434_T2 - Class 2 - DSC: 0.9666
✅ 0434_T2 - Class 3 - DSC: 0.9651
✅ 0434_T2 - Class 4 - DSC: 0.9678
✅ 0435_T1 - Class 1 - DSC: 0.9491
✅ 0435_T1 - Class 2 - DSC: 0.9499
✅ 0435_T1 - Class 3 - DSC: 0.9388
✅ 0435_T1 - Class 4 - DSC: 0.9324
✅ 0435_T2 - Class 1 - DSC: 0.9523
✅ 0435_T2 - Class 2 - DSC: 0.9483
✅ 0435_T2 - Class 3 - DSC: 0.9275
✅ 0435_T2 - Class 4 - DSC: 0.9353
✅ 0436_T1 - Class 1 - DSC: 0.9447
✅ 0436_T1 - Class 2 - DSC: 0.9550
✅ 0436_T1 - Class 3 - DSC: 0.9695
✅ 0436_T1 - Class 4 - DSC: 0.9648
✅ 0436_T2 - Class 1 - DSC: 0.9588
✅ 0436_T2 - Class 2 - DSC: 0.9561
✅ 0436_T2 - Class 3 - DSC: 0.9693
✅ 0436_T2 - Class 4 - DSC: 0.9624
✅ 0437_T1 - Class 1 - DSC: 0.9661
✅ 0437_T1 - Class 2 - DSC: 0.9639
✅ 0437_T1 - Class 3 - DSC: 0.9677
✅ 0437_T1 - Class 4 - DSC: 0.9713
✅ 0437_T2 - Class 1 - DSC: 0.9611
✅ 0437_T2 - Class 2 - DSC: 0.9637
✅ 0437_T2 - Class 3 - DSC: 0.8904
✅ 0437_T2 - Class 4 - DSC: 0.9299
✅ 0438_T1 - Class 1 - DSC: 0.9603
✅ 0438_T1 - Class 2 - DSC: 0.9674
✅ 0438_T1 - Cl

✅ 0486_T2 - Class 2 - DSC: 0.9688
✅ 0486_T2 - Class 3 - DSC: 0.9763
✅ 0486_T2 - Class 4 - DSC: 0.9745
✅ 0488_T1 - Class 1 - DSC: 0.8517
✅ 0488_T1 - Class 2 - DSC: 0.7948
✅ 0488_T1 - Class 3 - DSC: 0.9410
✅ 0488_T1 - Class 4 - DSC: 0.9116
✅ 0488_T2 - Class 1 - DSC: 0.9426
✅ 0488_T2 - Class 2 - DSC: 0.9488
✅ 0488_T2 - Class 3 - DSC: 0.9481
✅ 0488_T2 - Class 4 - DSC: 0.9294
✅ 0492_T1 - Class 1 - DSC: 0.9449
✅ 0492_T1 - Class 2 - DSC: 0.9540
✅ 0492_T1 - Class 3 - DSC: 0.9573
✅ 0492_T1 - Class 4 - DSC: 0.9513
✅ 0492_T2 - Class 1 - DSC: 0.9313
✅ 0492_T2 - Class 2 - DSC: 0.9364
✅ 0492_T2 - Class 3 - DSC: 0.9628
✅ 0492_T2 - Class 4 - DSC: 0.9698
✅ 0493_T1 - Class 1 - DSC: 0.9043
✅ 0493_T1 - Class 2 - DSC: 0.9348
✅ 0493_T1 - Class 3 - DSC: 0.9623
✅ 0493_T1 - Class 4 - DSC: 0.9471
✅ 0493_T2 - Class 1 - DSC: 0.9521
✅ 0493_T2 - Class 2 - DSC: 0.9677
✅ 0493_T2 - Class 3 - DSC: 0.9738
✅ 0493_T2 - Class 4 - DSC: 0.9699
✅ 0494_T1 - Class 1 - DSC: 0.9599
✅ 0494_T1 - Class 2 - DSC: 0.9134
✅ 0494_T1 - Cl

✅ 0528_T2 - Class 3 - DSC: 0.9731
✅ 0528_T2 - Class 4 - DSC: 0.9718
✅ 0529_T1 - Class 1 - DSC: 0.9530
✅ 0529_T1 - Class 2 - DSC: 0.9546
✅ 0529_T1 - Class 3 - DSC: 0.9643
✅ 0529_T1 - Class 4 - DSC: 0.9619
✅ 0529_T2 - Class 1 - DSC: 0.9512
✅ 0529_T2 - Class 2 - DSC: 0.9456
✅ 0529_T2 - Class 3 - DSC: 0.9035
✅ 0529_T2 - Class 4 - DSC: 0.9236
✅ 0531_T1 - Class 1 - DSC: 0.9566
✅ 0531_T1 - Class 2 - DSC: 0.9545
✅ 0531_T1 - Class 3 - DSC: 0.9627
✅ 0531_T1 - Class 4 - DSC: 0.9613
✅ 0531_T2 - Class 1 - DSC: 0.9737
✅ 0531_T2 - Class 2 - DSC: 0.9659
✅ 0531_T2 - Class 3 - DSC: 0.9608
✅ 0531_T2 - Class 4 - DSC: 0.9650
✅ 0532_T1 - Class 1 - DSC: 0.9311
✅ 0532_T1 - Class 2 - DSC: 0.9566
✅ 0532_T1 - Class 3 - DSC: 0.9366
✅ 0532_T1 - Class 4 - DSC: 0.9154
✅ 0532_T2 - Class 1 - DSC: 0.9287
✅ 0532_T2 - Class 2 - DSC: 0.9541
✅ 0532_T2 - Class 3 - DSC: 0.9573
✅ 0532_T2 - Class 4 - DSC: 0.9378
✅ 0533_T1 - Class 1 - DSC: 0.9533
✅ 0533_T1 - Class 2 - DSC: 0.9627
✅ 0533_T1 - Class 3 - DSC: 0.9587
✅ 0533_T1 - Cl

✅ 0569_T2 - Class 3 - DSC: 0.9572
✅ 0569_T2 - Class 4 - DSC: 0.9545
✅ 0570_T1 - Class 1 - DSC: 0.9467
✅ 0570_T1 - Class 2 - DSC: 0.9691
✅ 0570_T1 - Class 3 - DSC: 0.9642
✅ 0570_T1 - Class 4 - DSC: 0.9594
✅ 0570_T2 - Class 1 - DSC: 0.9487
✅ 0570_T2 - Class 2 - DSC: 0.9557
✅ 0570_T2 - Class 3 - DSC: 0.9558
✅ 0570_T2 - Class 4 - DSC: 0.9441
✅ 0571_T1 - Class 1 - DSC: 0.9220
✅ 0571_T1 - Class 2 - DSC: 0.8961
✅ 0571_T1 - Class 3 - DSC: 0.9031
✅ 0571_T1 - Class 4 - DSC: 0.8979
✅ 0571_T2 - Class 1 - DSC: 0.9205
✅ 0571_T2 - Class 2 - DSC: 0.9052
✅ 0571_T2 - Class 3 - DSC: 0.9263
✅ 0571_T2 - Class 4 - DSC: 0.9295
✅ 0572_T1 - Class 1 - DSC: 0.9305
✅ 0572_T1 - Class 2 - DSC: 0.9412
✅ 0572_T1 - Class 3 - DSC: 0.9019
✅ 0572_T1 - Class 4 - DSC: 0.9417
✅ 0572_T2 - Class 1 - DSC: 0.9543
✅ 0572_T2 - Class 2 - DSC: 0.9549
✅ 0572_T2 - Class 3 - DSC: 0.9422
✅ 0572_T2 - Class 4 - DSC: 0.9665
✅ 0573_T1 - Class 1 - DSC: 0.9493
✅ 0573_T1 - Class 2 - DSC: 0.9538
✅ 0573_T1 - Class 3 - DSC: 0.9638
✅ 0573_T1 - Cl

In [23]:
df = pd.DataFrame(records)
print(f"\n✅ Total pids evaluated: {len(df)}")
df['decrease_all'] = (df['decrease_class_1']) & (df['decrease_class_2'])  # 逻辑与
df_sorted = df.sort_values(
    by=['decrease_all', 'mean_dsc'],
    ascending=[False, False]
).reset_index(drop=True)

df_sorted




✅ Total pids evaluated: 1221


,pid,dsc_class_1,dsc_class_2,mean_dsc,decrease_class_1,decrease_class_2,decrease_all
0,0151_T1,0.9957,0.9958,0.9958,True,True,True
1,0033_T2,0.9957,0.9956,0.9957,True,True,True
2,0101_T2,0.9954,0.9958,0.9956,True,True,True
3,0151_T2,0.9955,0.9957,0.9956,True,True,True
4,0177_T2,0.9956,0.9956,0.9956,True,True,True
...,...,...,...,...,...,...,...
1216,s1424,0.8060,0.8893,0.8476,False,False,False
1217,word_0053,0.8783,0.8149,0.8466,False,False,False
1218,s1207,0.7948,0.8413,0.8181,False,False,False
1219,s1249,0.7631,0.8442,0.8036,False,False,False


In [24]:
def assign_dataset(pid):
    if 'ID' in pid:
        return 'bedrest'
    elif 'T1' in pid:
        return 'T1W'
    elif 'T2' in pid:
        return 'T2W'
    elif pid.startswith('s'):
        return 'ct_tt'
    elif pid.startswith('word'):
        return 'ct_word'
    else:
        return 'AFL'
df_sorted['dataset'] = df_sorted['pid'].apply(assign_dataset)
df_sorted

,pid,dsc_class_1,dsc_class_2,mean_dsc,decrease_class_1,decrease_class_2,decrease_all,dataset
0,0151_T1,0.9957,0.9958,0.9958,True,True,True,T1W
1,0033_T2,0.9957,0.9956,0.9957,True,True,True,T2W
2,0101_T2,0.9954,0.9958,0.9956,True,True,True,T2W
3,0151_T2,0.9955,0.9957,0.9956,True,True,True,T2W
4,0177_T2,0.9956,0.9956,0.9956,True,True,True,T2W
...,...,...,...,...,...,...,...,...
1216,s1424,0.8060,0.8893,0.8476,False,False,False,ct_tt
1217,word_0053,0.8783,0.8149,0.8466,False,False,False,ct_word
1218,s1207,0.7948,0.8413,0.8181,False,False,False,ct_tt
1219,s1249,0.7631,0.8442,0.8036,False,False,False,ct_tt


In [25]:
# Ensure decrease_all column exists
df['decrease_all'] = df['decrease_class_1'] & df['decrease_class_2']

# Apply assign_dataset if needed
if 'dataset' not in df.columns:
    df['dataset'] = df['pid'].apply(assign_dataset)

# Prepare a list to collect results from each dataset
df_top_list = []

# Group by dataset
for dataset, group in df.groupby('dataset'):
    # Filter where decrease_all is True
    df_true = group[group['decrease_all'] == True]
    df_true = df_true[df_true['mean_dsc'] > 0.95]
    # Determine how many rows to keep (10% or at least 2)
    top_n = max(2, int(len(group) * 0.2))
    
    # Sort and keep top_n rows by mean_dsc
    df_top = df_true.sort_values(by='mean_dsc', ascending=False).head(top_n)
    
    df_top_list.append(df_top)

# Concatenate all top selections
df_top_all = pd.concat(df_top_list).reset_index(drop=True)

df_top_all


,pid,dsc_class_1,dsc_class_2,mean_dsc,decrease_class_1,decrease_class_2,decrease_all,dataset
0,GL_110,0.9822,0.9800,0.9811,True,True,True,AFL
1,GL_118,0.9824,0.9768,0.9796,True,True,True,AFL
2,Jake_139,0.9804,0.9740,0.9772,True,True,True,AFL
3,GL_116,0.9761,0.9781,0.9771,True,True,True,AFL
4,Jake_144,0.9760,0.9740,0.9750,True,True,True,AFL
...,...,...,...,...,...,...,...,...
224,word_0063,0.9610,0.9681,0.9646,True,True,True,ct_word
225,word_0055,0.9619,0.9628,0.9624,True,True,True,ct_word
226,word_0113,0.9565,0.9664,0.9615,True,True,True,ct_word
227,word_0009,0.9483,0.9637,0.9560,True,True,True,ct_word


In [26]:
for _, row in df_top_all.iterrows():
    print(f"pid: {row['pid']}, "
          f"mean_dsc: {row['mean_dsc']:.4f}, "
          f"decrease_class_1: {row['decrease_class_1']}, "
          f"decrease_class_2: {row['decrease_class_2']}, "
          f"decrease_all: {row['decrease_all']}, "
          f"dataset: {row['dataset']} \n")


pid: GL_110, mean_dsc: 0.9811, decrease_class_1: True, decrease_class_2: True, decrease_all: True, dataset: AFL 

pid: GL_118, mean_dsc: 0.9796, decrease_class_1: True, decrease_class_2: True, decrease_all: True, dataset: AFL 

pid: Jake_139, mean_dsc: 0.9772, decrease_class_1: True, decrease_class_2: True, decrease_all: True, dataset: AFL 

pid: GL_116, mean_dsc: 0.9771, decrease_class_1: True, decrease_class_2: True, decrease_all: True, dataset: AFL 

pid: Jake_144, mean_dsc: 0.9750, decrease_class_1: True, decrease_class_2: True, decrease_all: True, dataset: AFL 

pid: GL_112, mean_dsc: 0.9738, decrease_class_1: True, decrease_class_2: True, decrease_all: True, dataset: AFL 

pid: Jake_136, mean_dsc: 0.9725, decrease_class_1: True, decrease_class_2: True, decrease_all: True, dataset: AFL 

pid: 0151_T1, mean_dsc: 0.9958, decrease_class_1: True, decrease_class_2: True, decrease_all: True, dataset: T1W 

pid: 0308_T1, mean_dsc: 0.9955, decrease_class_1: True, decrease_class_2: True, d

In [27]:
import os
import glob
import shutil

# Directories
img_dir = '/home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_test_data/test_img_in_nii'
seg_dir = '/home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw_912_curated'

out_img_dir = '/home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_train_data_raw/MRI_highIOU_2class_nnsam2_stage2/img_in_nii_L4-L5'
out_seg_dir = '/home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_train_data_raw/MRI_highIOU_2class_nnsam2_stage2/seg_in_nii_L4-L5'


out_img_dir = '/home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_train_data_raw/MRI_highIOU_2class_nnsam2_stage3/img_in_nii_L4-L5'
out_seg_dir = '/home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_train_data_raw/MRI_highIOU_2class_nnsam2_stage3/seg_in_nii_L4-L5'

# Create output dirs if not exist
os.makedirs(out_img_dir, exist_ok=True)
os.makedirs(out_seg_dir, exist_ok=True)

# Iterate and copy
for pid in df_top_all['pid']:
    # Use glob to find matching files
    img_matches = glob.glob(os.path.join(img_dir, f"{pid}*.nii*"))
    seg_matches = glob.glob(os.path.join(seg_dir, f"{pid}*.nii*"))

    # Check and copy image
    if len(img_matches) == 1:
        shutil.copy(img_matches[0], out_img_dir)
        print(f"Copied image: {img_matches[0]}")
    elif len(img_matches) == 0:
        print(f"[Missing] Image not found for {pid}")
    else:
        print(f"[Warning] Multiple image matches for {pid}: {img_matches}")

    # Check and copy segmentation
    if len(seg_matches) == 1:
        shutil.copy(seg_matches[0], out_seg_dir)
        print(f"Copied seg: {seg_matches[0]}")
    elif len(seg_matches) == 0:
        print(f"[Missing] Segmentation not found for {pid}")
    else:
        print(f"[Warning] Multiple seg matches for {pid}: {seg_matches}")


Copied image: /home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_test_data/test_img_in_nii/GL_110_L4L5_img_nnunet_to_seg_0000.nii.gz
Copied seg: /home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw_912_curated/GL_110_L4L5_img_nnunet_to_seg.nii.gz
Copied image: /home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_test_data/test_img_in_nii/GL_118_L4L5_img_nnunet_to_seg_0000.nii.gz
Copied seg: /home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw_912_curated/GL_118_L4L5_img_nnunet_to_seg.nii.gz
Copied image: /home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_test_data/test_img_in_nii/Jake_139_L4L5_img_nnunet_to_seg_0000.nii.gz
Copied seg: /home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii_raw_912_curated/Jake_139_L4L5_img_nnunet_to_seg.nii.gz
Copied image: /home/zhongyi/Desktop/nnunetv1/nnUNet_raw_data_base/nnUNet_test_data/test_img_in_nii/GL_116_L4L5_img_nnunet_t